http://scikit-learn.org/stable/auto_examples/ensemble/plot_gradient_boosting_quantile.html

In [2]:
import numpy as np
from scipy.stats import binom_test
from sklearn.base import BaseEstimator, RegressorMixin
from xgboost.sklearn import XGBRegressor
from functools import partial
class XGBOOSTQUANTILE(BaseEstimator, RegressorMixin):
    def __init__(self, quant_alpha,quant_delta,quant_thres,quant_var,
    n_estimators = 100,max_depth = 3,reg_alpha = 5.,reg_lambda=1.0,gamma=0.5):
        self.quant_alpha = quant_alpha
        self.quant_delta = quant_delta 
        self.quant_thres = quant_thres 
        self.quant_var = quant_var 
        #xgboost parameters 
        self.n_estimators = n_estimators 
        self.max_depth = max_depth 
        self.reg_alpha= reg_alpha 
        self.reg_lambda = reg_lambda 
        self.gamma = gamma 
        #keep xgboost estimator in memory 
        self.clf = None 
    def fit(self, X, y): 
        def quantile_loss(y_true, y_pred,_alpha,_delta,_threshold,_var): 
            x = y_true - y_pred 
            grad = (x<(_alpha-1.0)*_delta)*(1.0-_alpha)- ((x>=(_alpha-1.0)*_delta)&
                                    (x<_alpha*_delta) )*x/_delta-_alpha*(x>_alpha*_delta) 
            hess = ((x>=(_alpha-1.0)*_delta)& (x<_alpha*_delta) )/_delta 
            _len = np.array([y_true]).size 
            var = (2*np.random.randint(2, size=_len)-1.0)*_var 
            grad = (np.abs(x)<_threshold )*grad - (np.abs(x)>=_threshold )*var 
            hess = (np.abs(x)<_threshold )*hess + (np.abs(x)>=_threshold ) 
            return grad, hess 
         self.clf = XGBRegressor(
         objective=partial( quantile_loss,
                            _alpha = self.quant_alpha,
                            _delta = self.quant_delta,
                            _threshold = self.quant_thres,
                            _var = self.quant_var), 
                            n_estimators = self.n_estimators,
                            max_depth = self.max_depth,
                            reg_alpha =self.reg_alpha, 
                            reg_lambda = self.reg_lambda,
                            gamma = self.gamma )
         self.clf.fit(X,y) 
         return self 
    def predict(self, X): 
        y_pred = self.clf.predict(X) 
        return y_pred 
    def score(self, X, y): 
        y_pred = self.clf.predict(X) 
        score = (self.quant_alpha-1.0)*(y-y_pred)*(y<y_pred)+self.quant_alpha*(y-y_pred)* (y>=y_pred) 
        score = 1./np.sum(score) 
        return score

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 32)

In [ ]:
import numpy as np
np.random.seed(1)
def f(x):
    """The function to predict."""
    return x * np.sin(x)
#----------------------------------------------------------------------
# First the noiseless case
X = np.atleast_2d(np.random.uniform(0, 10.0, size=100)).T
X = X.astype(np.float32)
# Observations
y = f(X).ravel()
dy = 1.5 + 1.0 * np.random.random(y.shape)
noise = np.random.normal(0, dy)
y += noise
y = y.astype(np.float32)